In [1]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import HTML

In [2]:
from bqplot import Figure, Scatter, Axis, LinearScale

In [3]:
EXPLANATION = """\
<div class="app-sidebar">
<p><em>Created at NeuroHackademy 2024.</em><p>

<p>Select what data to plot in the dropdowns of each section.</p>

<p>Created using the <a href="https://github.com/voila-dashboards/voici-demo">
Voici demo template repository</a>.</p>

</div>
"""

In [4]:
HTML("""\
<style>
.app-subtitle {
    font-size: 1.5em;
}

.app-subtitle a {
    color: #106ba3;
}

.app-subtitle a:hover {
    text-decoration: underline;
}

.app-sidebar p {
    margin-bottom: 1em;
    line-height: 1.7;
}

.app-sidebar a {
    color: #106ba3;
}

.app-sidebar a:hover {
    text-decoration: underline;
}
</style>
""")

In [9]:
class App:

    def __init__(self, df):
        self._df = df
        available_indicators = self._df['Indicator Name'].unique()
        self._x_dropdown = self._create_indicator_dropdown(available_indicators, 0)
        self._y_dropdown = self._create_indicator_dropdown(available_indicators, 1)

        x_scale = LinearScale()
        y_scale = LinearScale()

        self._x_axis = Axis(scale=x_scale, label="X")
        self._y_axis = Axis(scale=y_scale, orientation="vertical", label="Y")

        self._scatter = Scatter(
            x=[], y=[], scales={"x": x_scale, "y": y_scale}
        )

        self._figure = Figure(marks=[self._scatter], axes=[self._x_axis, self._y_axis], layout=dict(width="99%"), animation_duration=1000)

        self._year_slider, year_slider_box = self._create_year_slider(
            min(df['Year']), max(df['Year'])
        )
        _app_container = widgets.VBox([
            widgets.HBox([self._x_dropdown, self._y_dropdown]),
            self._figure,
            year_slider_box
        ], layout=widgets.Layout(align_items='center', flex='3 0 auto'))
        self.container = widgets.VBox([
            widgets.HTML(
                (
                    '<h1>Explore the Human Connectome Project Young Adult dataset!</h1>'
                    '<h2 class="app-subtitle"><a href="https://github.com/NeuroHackademy2024/neuro-nav">Link to code</a></h2>'
                ),
                layout=widgets.Layout(margin='0 0 5em 0')
            ),
            widgets.HBox([
                _app_container,
                widgets.HTML(EXPLANATION, layout=widgets.Layout(margin='0 0 0 2em'))
            ])
        ], layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))
        self._update_app()

    @classmethod
    def from_csv(cls, path):
        df = pd.read_csv(path)
        return cls(df)

    def _create_indicator_dropdown(self, indicators, initial_index):
        dropdown = widgets.Dropdown(options=indicators, value=indicators[initial_index])
        dropdown.observe(self._on_change, names=['value'])
        return dropdown

    def _create_year_slider(self, min_year, max_year):
        year_slider_label = widgets.Label('Year range: ')
        year_slider = widgets.IntRangeSlider(
            min=min_year, max=max_year,
            layout=widgets.Layout(width='500px'),
            continuous_update=False
        )
        year_slider.observe(self._on_change, names=['value'])
        year_slider_box = widgets.HBox([year_slider_label, year_slider])
        return year_slider, year_slider_box

    def _on_change(self, _):
        self._update_app()

    def _update_app(self):
        x_indicator = self._x_dropdown.value
        y_indicator = self._y_dropdown.value
        year_range = self._year_slider.value

        with self._scatter.hold_sync():
            df = self._df[self._df['Year'].between(*year_range)].dropna()
            x = df[df['Indicator Name'] == x_indicator]['Value']
            y = df[df['Indicator Name'] == y_indicator]['Value']

            self._x_axis.label = x_indicator
            self._y_axis.label = y_indicator

            self._scatter.default_opacities = [0.2]

            self._scatter.x = x
            self._scatter.y = y

In [56]:
# app = App.from_csv("indicators.csv")

# app.container

In [89]:
# Items that populate each section of the dashboard get defined here

# List of items in the Demographics section:
demogr_items = [
    widgets.HTML(('<h4>Demographics Information</h4>'), layout=widgets.Layout(width='auto', justify_content='space-between')), #section title
    # add more widgets here
]

# List of items in the Behavioural section:
behav_items = [
    widgets.HTML(('<h4>Behavioural Data</h4>'), layout=widgets.Layout(width='auto', justify_content='space-between')), #section title
    # add more widgets here
]

# List of items in the Neuroimaging section:
neuro_items = [
    widgets.HTML(('<h4>Neuroimaging Data</h4>'), layout=widgets.Layout(width='auto', justify_content='space-between')), #section title
    # add more widgets here
]

In [91]:
# Render the different sections of the app

# The header section contains only the main title and subtitle of the website as an HTML widget:
header           = widgets.HTML((
                        '<h1>Explore the Human Connectome Project Young Adult dataset!</h1>'
                        '<p>Select what data to plot in the dropdowns of each section</p>'), 
                        layout=widgets.Layout(width='auto', grid_area='header'))

# The demographics section is a "box" (container) that contains all of the widgets listed under the demogr_items (defined in cell above):
demographics     = widgets.VBox(children=demogr_items,
                        layout=widgets.Layout(width='auto', grid_area='demographics', border='2px solid grey', align_items='center'))

# The behavioural section is a "box" (container) that contains all of the widgets listed under the behav_items (defined in cell above):
behavioural      = widgets.VBox(children=behav_items,
                        layout=widgets.Layout(width='auto', grid_area='behavioural', border='2px solid grey', align_items='center'))

# The neuroimaging section is a "box" (container) that contains all of the widgets listed under the neuro_items (defined in cell above):
neuroimaging     = widgets.VBox(children=neuro_items,
                        layout=widgets.Layout(width='auto', grid_area='neuroimaging', border='2px solid grey', align_items='center'))

# The footer section contains only the footer string and related hyperlinks as an HTML widget:
footer           = widgets.HTML((
                        '<p><i>Created at NeuroHackademy 2024 using the <b style="color:blue"><a href="https://github.com/voila-dashboards/voici-demo"> Voici demo template repository</a></b>. | <b style="color:blue"><a href="https://github.com/NeuroHackademy2024/neuro-nav">Link to code</a></b></i><p>'),
                        layout=widgets.Layout(width='auto', grid_area='footer'))

widgets.GridBox(children=[header, demographics, behavioural, neuroimaging, footer],
        layout=widgets.Layout(
            width='100%',
            grid_template_rows='auto auto auto',
            grid_template_columns='25% 25% 25% 25%',
            grid_template_areas='''
            "header header header header"
            "demographics demographics behavioural behavioural"
            "neuroimaging neuroimaging neuroimaging neuroimaging"
            "footer footer footer footer"
            ''')
       )

GridBox(children=(HTML(value='<h1>Explore the Human Connectome Project Young Adult dataset!</h1><p>Select what…